<a href="https://colab.research.google.com/github/ajay47847/pcg_classification/blob/main/featureData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
from scipy.io import wavfile
from scipy.signal import butter, filtfilt
from scipy.stats import kurtosis, skew
import matplotlib.pyplot as plt

# Function to load audio files from a specified folder
def load_audio_files(file_list, folder_path):
    audio_data = []
    record_names = []
    min_length = float('inf')
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        sr, y = wavfile.read(file_path)  # Load the audio file
        audio_data.append(y)
        record_names.append(file_name)
        min_length = min(min_length, len(y))
    # Trim all audio files to the minimum length
    audio_data = [audio[:min_length] for audio in audio_data]
    return np.array(audio_data), record_names, sr

# Function to apply a bandpass filter
def apply_filter(data, sample_rate, filter_type, cutoff_freqs):
    nyquist = 0.5 * sample_rate
    normalized_cutoff = [freq / nyquist for freq in cutoff_freqs]
    b, a = butter(N=1, Wn=normalized_cutoff, btype=filter_type)
    filtered_data = filtfilt(b, a, data)
    return filtered_data

# Function to normalize audio data using z-score
def z_score_normalize_audio_data(audio_data):
    normalized_audio = []
    for signal in audio_data:
        mean_val = np.mean(signal)
        std_val = np.std(signal)
        if std_val == 0:
            normalized_signal = signal - mean_val
        else:
            normalized_signal = (signal - mean_val) / std_val
        normalized_audio.append(normalized_signal)
    return np.array(normalized_audio)

# Feature extraction functions
def calculate_zero_crossing_rate(audio_data):
    zero_crossing_rates = []
    for signal in audio_data:
        zsr = np.sum(np.abs(np.diff(np.sign(signal)))) / len(signal)
        zero_crossing_rates.append(zsr)
    return np.array(zero_crossing_rates)

def calculate_kurtosis(audio_data):
    kurtosis_values = []
    for signal in audio_data:
        kurtosis_values.append(kurtosis(signal))
    return np.array(kurtosis_values)

def calculate_skewness(audio_data):
    skewness_values = []
    for signal in audio_data:
        skewness_values.append(skew(signal))
    return np.array(skewness_values)

def compute_psd(audio_data, sample_rate):
    psd_list = []
    for signal in audio_data:
        frequencies, psd = welch(signal, sample_rate)
        psd_list.append(psd)
    psd_array = np.array(psd_list)
    return frequencies, psd_array

# Function to slice the PSD and retrieve the values
def slice_psd(psd_array, frequencies, start_freq, stop_freq, increment, overlap):
    psd_slices = []
    low = start_freq
    while low < stop_freq:
        high = low + increment
        if high > stop_freq:
            high = stop_freq
        freq_mask = (frequencies >= low) & (frequencies < high)
        sliced_psd = psd_array[:, freq_mask]
        psd_slices.append((low, high, sliced_psd))
        low += (increment - overlap)
    return psd_slices

# Load the Excel file and select relevant columns
df = pd.read_excel('/content/PMEA_37_12_2181_OnlineAppendix.xlsx', sheet_name='SUAHSDB')
df = df[['Record name', 'Type (-1=normal 1=abnormal)']]

# Define the folder path containing the audio files
folder_path = '/content/drive/MyDrive/training-f'

# Get a list of all .wav files in the folder
audio_files = [file for file in os.listdir(folder_path) if file.endswith('.wav')]

# Extract 2.5 seconds of audio
duration = 2.5
num_samples = int(duration * sample_rate)
audio_data = audio_data[:, :num_samples]

# Apply bandpass filter
audio_data_bandpass = apply_filter(audio_data, sample_rate, 'band', [25, 600])

# Z-normalize the audio data
audio_data_norm = z_score_normalize_audio_data(audio_data_bandpass)

# Extract features
zero_crossing_rates = calculate_zero_crossing_rate(audio_data_norm)
kurtosis_values = calculate_kurtosis(audio_data_norm)
skewness_values = calculate_skewness(audio_data_norm)

# Compute PSD
frequencies, psd_array = compute_psd(audio_data_norm, sample_rate)

# Define slicing parameters
start_freq = 250
stop_freq = 600
increment = 50
overlap = 25

# Slice PSD and get values
psd_slices = slice_psd(psd_array, frequencies, start_freq, stop_freq, increment, overlap)

# Combine basic features and PSD slices into a single DataFrame
columns = ['Zero Crossing Rate', 'Kurtosis', 'Skewness']
for low, high, _ in psd_slices:
    columns.append(f'PSD {low}-{high} Hz')

# Create DataFrame for features with record names and types
df_features = pd.DataFrame({
    'Record Name': record_names_loaded,
    'Zero Crossing Rate': zero_crossing_rates,
    'Kurtosis': kurtosis_values,
    'Skewness': skewness_values,
})



# Add PSD slices to the DataFrame
for low, high, psd_slice in psd_slices:
    df_features[f'PSD {low}-{high} Hz'] = np.mean(psd_slice, axis=1)

# Sort DataFrame by 'Record Name' in ascending order and reset index
df_features_sorted = df_features.sort_values(by='Record Name').reset_index(drop=True)

# Reassign the 'Type' column to maintain its original order
df_features_sorted['Type (-1=normal 1=abnormal)'] = df['Type (-1=normal 1=abnormal)']


# Print the sorted DataFrame with the updated index
print(df_features_sorted.to_string())


    Record Name  Zero Crossing Rate   Kurtosis  Skewness  PSD 250-300 Hz  PSD 275-325 Hz  PSD 300-350 Hz  PSD 325-375 Hz  PSD 350-400 Hz  PSD 375-425 Hz  PSD 400-450 Hz  PSD 425-475 Hz  PSD 450-500 Hz  PSD 475-525 Hz  PSD 500-550 Hz  PSD 525-575 Hz  PSD 550-600 Hz  PSD 575-600 Hz  Type (-1=normal 1=abnormal)
0     f0001.wav              0.5604  22.793595  0.393832        0.001035        0.000469        0.000214        0.000230        0.000282        0.000234        0.000155        0.000107        0.000115        0.000166    1.948934e-04    2.216737e-04    2.481510e-04    2.676894e-04                           -1
1     f0002.wav              0.5504   9.824735  0.044023        0.000412        0.000408        0.000432        0.000393        0.000716        0.001477        0.002525        0.002729        0.002596        0.002828    2.866852e-03    2.316540e-03    2.096960e-03    2.249682e-03                           -1
2     f0003.wav              0.3636  21.751029  0.320183        0.0025